# Проверка работы Word2Vec

### Настраиваем корневую папку

In [1]:
import first_script

### Импорт нужных библиотек

In [2]:
from model import Word2Vec
from tokenizer import AutoTokenizer
import os

### Загружаем текст на котором будем обучать модель

In [3]:
# text = "Мама мыла раму, пока кошка ходила по улице, а папа ел хлеб. Дети гуляли по улице!"
text = ""
with open('../data/voina_i_mir.txt', 'r') as f:
    text = f.read()

### Загружаем предобученый токенизатор

In [4]:
tokenizer = AutoTokenizer("../tokenizer/tokenizer.bin")

### Превращаем текст в индексы

In [5]:
text_idxs = tokenizer.encode(text, add_special_tokens=True)

### Параметры модели

In [6]:
embed_dim = 256
window_size = 20
device = 'cuda'

### Параметры обучения

In [7]:
num_epochs = 10
batch_size = 2500
num_workers = os.cpu_count()

### Загружаем модель и обучаем

In [8]:
model = Word2Vec(embed_dim, window_size, len(tokenizer.words), device=device)
res = model.fit(text_idxs, batch_size, num_epochs, num_workers=num_workers)

Epoch [1/10, time: 1.171 minutes]:
                            Train Loss 14.246462, Train Perplexity 1538720.927
                            Test Loss 13.235621, Test Perplexity 559960.123
Epoch [2/10, time: 2.347 minutes]:
                            Train Loss 11.329529, Train Perplexity 83243.797
                            Test Loss 11.326873, Test Perplexity 83023.023
Epoch [3/10, time: 3.556 minutes]:
                            Train Loss 9.861915, Train Perplexity 19185.593
                            Test Loss 10.410132, Test Perplexity 33194.247
Epoch [4/10, time: 4.719 minutes]:
                            Train Loss 9.072693, Train Perplexity 8714.055
                            Test Loss 9.911676, Test Perplexity 20164.437
Epoch [5/10, time: 5.869 minutes]:
                            Train Loss 8.591814, Train Perplexity 5387.379
                            Test Loss 9.624417, Test Perplexity 15129.731
Epoch [6/10, time: 7.022 minutes]:
                            Train 

### Тестируем метод K_Nearest: ищет k ближайших слов на основе эмбеддингов

Гиперпараметр use_cosine отвечает за метод поиска расстояния между эмбеддингами. В случае use_cosine=True используется косинусовое расстояние (нормировано от 0 до 1). Иначе - евклидово расстояние

Гиперпараметр k - кол-во ближайших слов, которые выводятся. Замечу, что первое слово - само слово, по которому искали ближайшие слова. Выводится для удобства.

In [9]:
word1 = 'Андрей'
word1_idx = tokenizer.encode(word1)
near1 = model.k_Nearest(word1_idx, k=10, use_cosine=True)
print(tokenizer.decode(near1))

['Андрей', 'придвигал', 'повезти', 'победившего', 'оставить', 'защищать', 'очертаний', 'перилы', 'старинная', 'Безухову', 'писали']


In [10]:
word1 = 'Андрей'
word1_idx = tokenizer.encode(word1)
near1 = model.k_Nearest(word1_idx, k=10, use_cosine=False)
print(tokenizer.decode(near1))

['Андрей', 'державшему', 'невеста', 'победившего', 'молодому', 'очертаний', 'волка', 'вечерни', 'распоряжениям', 'препятствуют', 'оставить']


Гиперпараметр return_embed выводит эмбеддинги данных слов, в случае если return_embed=True, иначе не выводит

In [17]:
word1 = 'добрый'
word1_idx = tokenizer.encode(word1)
near1, embed = model.k_Nearest(word1_idx, k=10, return_embed=True)
print(tokenizer.decode(near1), embed.shape)

['добрый', 'сходи', 'роты', 'подав', 'скучный', 'Филями', 'Mon', 'молоток', 'по', 'Voyons', 'прибора'] torch.Size([11, 256])


### Одна из главных особенностей модели: удобные и крутые визуализации!

#### Визуализируем метрики обучения

In [18]:
Word2Vec.plot_metrics(*res)

#### Визуализируем слова на двумерной плоскости на основе их эмбеддингов

Возьмем слово добрый и его 10 ближайших слов

In [19]:
Word2Vec.plot_embeddings(tokenizer.decode(near1), embed)

Добавим слово которое не является ближайшим словом к данному. Посмотрим где оно будет отображено

In [14]:
import torch

In [20]:
fake_word = 'злой'
fake_word_idx = tokenizer.encode(fake_word)
fake_embed = model[fake_word_idx]

fake_embed = torch.cat((embed, fake_embed), dim=0)
fake_near = tokenizer.decode(near1) + [fake_word]
Word2Vec.plot_embeddings(fake_near, fake_embed)

: 

Как видим, оно действительно дальше всех! Интересно, что все слова слева от слова добрый...